# list_unused_code_environments
This has been tested and validated on Dataiku DSS v14.1.2, running on Alma and Ubuntu Linux. It should work with the "Python 3 (ipykernel)" built-in code environment.

Written by Tim H - Sept 2025

In [0]:
import shutil
import glob
import os
import dataiku
from concurrent.futures import ThreadPoolExecutor

# Configuration

In [0]:
client = dataiku.api_client()
code_envs = client.list_code_envs()

code_env_usages = {}

In [0]:
def _process_code_env(code_env_info):
    try:
        client = dataiku.api_client()
        envName = code_env_info['envName']
        code_env = client.get_code_env(code_env_info['envLang'], envName)

        # print(f'Starting rebuilding {envName} ...')
        # env_path = os.path.join('/data/dataiku/dss_data/code-envs/python', envName)

        # rebuild it
        res = code_env.list_usages()
        code_env_usages[envName] = 


    except Exception as e:
        try:
            if not force_rebuild_env:
                # print(f'Failed to build {envName} without force rebuild, trying again with force rebuild...')
                res = code_env.update_packages(force_rebuild_env=True)
                print(f'Success: {envName} when Force rebuild')
                pass
        except Exception as e:
            print(f"FAILED: {envName}, even with force rebuild")
            failed_builds.add(envName) # potential bug where this doesn't happen, should use a finally clause
            pass

In [0]:
# >2 multi-threading causes /tmp to balloon in disk space from pip, causes / root file system to have 0% free space
# Fleet Manager instances only have a root file system size of 31 GB, 21 of which are used by base OS
# /tmp is mounted to /
clear_pip_tmp() # in case I stopped the last run
max_workers = 1 # os.cpu_count() or 1
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    executor.map(_process_code_env, code_envs)
    
print('\nFinished.')
clear_pip_tmp() # clean up

In [0]:
failed_builds

In [0]:
successful_builds